In [1]:
import tensorflow as tf
import numpy as np

In [2]:
train_file = open("../../data/iyer/train.txt")

In [3]:
train_file_contents = train_file.readlines()

In [4]:
len(train_file_contents)

52997

In [5]:
train = []
for line in train_file_contents:
    items = line.split('\t')
    if len(items) == 5:
        train.append(line.split('\t')[2])

In [6]:
len(train)

52795

In [7]:
len(train[0])

31

In [8]:
len(train[1])

23

In [9]:
import sys
sys.path.append("../../src")
from text_data_utils import *
from os import path
import gensim

In [10]:
train_tok = tokenize_texts(train)

In [11]:
wv = gensim.models.Word2Vec(train_tok).wv

In [12]:
max_len = max(len(text) for text in train_tok)

In [13]:
train_tensor = tokenized_texts_to_tensor(train_tok, wv, max_len)

In [14]:
train_tensor[0, 0]

array([ 2.40965456e-01, -1.68591738e-01, -1.21166356e-01, -7.00770020e-02,
        5.42261541e-01, -1.90423921e-01, -2.93175071e-01, -3.36992919e-01,
       -4.43560123e-01, -8.06656405e-02,  9.69258785e-01,  5.79911768e-01,
        3.59580606e-01,  1.58664152e-01, -7.05792129e-01,  4.06532168e-01,
       -6.12411857e-01,  1.09886892e-01,  2.16908717e+00, -2.16471687e-01,
       -2.39513546e-01,  9.52519476e-01, -5.05990721e-02,  8.70549202e-01,
       -9.07646000e-01, -5.23253977e-02, -6.07500911e-01,  1.13252950e+00,
        9.25281227e-01,  5.35802916e-02, -2.36804366e-01, -9.57369626e-01,
        4.25008565e-01,  7.79695153e-01, -4.74927723e-01,  5.10380507e-01,
        2.22434029e-01, -7.18683600e-02, -1.13020885e+00,  8.24752092e-01,
       -6.55958876e-02, -2.18344495e-01, -4.31710213e-01, -5.30911982e-01,
        3.63499045e-01, -9.79624540e-02, -1.78358674e+00,  9.40221012e-01,
       -4.70870584e-01,  1.10352505e-02,  1.74193934e-01,  5.21314383e-01,
       -4.24552917e-01, -

In [15]:
train_tensor_fl = np.reshape(train_tensor, (train_tensor.shape[0], train_tensor.shape[1] * train_tensor.shape[2]))

In [16]:
train_tensor.shape

(52795, 39, 100)

In [17]:
train_tensor_fl.shape

(52795, 3900)

In [18]:
val_file_contents = open('../../data/iyer/valid.txt').readlines()
val = []
for line in val_file_contents:
    items = line.split('\t')
    if len(items) == 5:
        val.append(line.split('\t')[2])
val_tok = tokenize_texts(val)
val_tensor = tokenized_texts_to_tensor(val_tok, wv, max_len)
val_tensor_fl = np.reshape(val_tensor, (val_tensor.shape[0], val_tensor.shape[1] * val_tensor.shape[2]))

In [19]:
from mlp_vae import MLPVariationalAutoEncoder

In [20]:
latent_dim = 256

In [21]:
model = MLPVariationalAutoEncoder(train_tensor_fl.shape[1], latent_dim, [1024, 512], final_activation='linear')
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(train_tensor_fl, train_tensor_fl, batch_size=512, epochs=25, verbose=1, shuffle=True,
                    validation_data=(val_tensor_fl, val_tensor_fl))

Train on 52795 samples, validate on 6599 samples
Epoch 1/25
52795/52795 [==============================] - 8s 152us/sample - loss: 0.0893 - val_loss: 0.0599
Epoch 2/25
52795/52795 [==============================] - 5s 101us/sample - loss: 0.0526 - val_loss: 0.0473
Epoch 3/25
52795/52795 [==============================] - 5s 104us/sample - loss: 0.0428 - val_loss: 0.0413
Epoch 4/25
52795/52795 [==============================] - 6s 110us/sample - loss: 0.0376 - val_loss: 0.0370
Epoch 5/25
52795/52795 [==============================] - 5s 101us/sample - loss: 0.0341 - val_loss: 0.0343
Epoch 6/25
52795/52795 [==============================] - 5s 102us/sample - loss: 0.0312 - val_loss: 0.0324
Epoch 7/25
52795/52795 [==============================] - 5s 104us/sample - loss: 0.0292 - val_loss: 0.0300
Epoch 8/25
52795/52795 [==============================] - 5s 99us/sample - loss: 0.0275 - val_loss: 0.0280
Epoch 9/25
52795/52795 [==============================] - 5s 100us/sample - loss: 0.0256

In [22]:
import random

In [23]:
random.seed()
random_idx = random.randrange(train_tensor.shape[0])

In [24]:
print("(Training Set) Input: ", tensor_to_tokenized_texts(np.array([train_tensor[random_idx]]), wv)[0])

(Training Set) Input:  ['<s>', 'What', 'is', 'correct', 'way', 'to', 'a', 'remove', 'few', 'characters', 'at', 'the', 'end', 'of', 'string', 'C', '#', '</s>']


In [25]:
rec = np.reshape(model.predict(np.array([train_tensor_fl[random_idx]])), (1, train_tensor.shape[1], train_tensor.shape[2]))

In [26]:
print("(Training Set) Reconstructed: ", tensor_to_tokenized_texts(rec, wv)[0])

(Training Set) Reconstructed:  ['<s>', 'What', 'is', 'any', 'way', 'to', 'a', 'strings', 'reverse', 'characters', 'at', 'the', 'end', 'of', 'string', 'C', '#', '</s>']


In [27]:
wv.similar_by_vector(rec[0, 9])

[('characters', 0.8964152336120605),
 ('numbers', 0.8511375188827515),
 ('regex', 0.8438798189163208),
 ('strings', 0.8184096813201904),
 ('words', 0.8077085614204407),
 ('char', 0.8045477867126465),
 ('Regex', 0.7984655499458313),
 ('comma', 0.7961603403091431),
 ('string', 0.7829160690307617),
 ('character', 0.777494490146637)]

In [28]:
test_file_contents = open('../../data/iyer/test.txt').readlines()
test = []
for line in test_file_contents:
    items = line.split('\t')
    if len(items) == 5:
        test.append(line.split('\t')[2])
test_tok = tokenize_texts(test)
test_tensor = tokenized_texts_to_tensor(test_tok, wv, max_len)
test_tensor_fl = np.reshape(test_tensor, (test_tensor.shape[0], test_tensor.shape[1] * test_tensor.shape[2]))

In [29]:
model.evaluate(test_tensor_fl, test_tensor_fl, verbose=0)

0.045086646639121125

In [30]:
random.seed()
random_idx = random.randrange(test_tensor.shape[0])

In [31]:
print("(Test Set) Input: ", tensor_to_tokenized_texts(np.array([test_tensor[random_idx]]), wv)[0])

(Test Set) Input:  ['<s>', 'detect', '<UNK>', 'when', 'copy', 'big', 'text', 'data', 'into', 'textbox', '</s>']


In [32]:
print("(Test Set) Reconstructed: ", tensor_to_tokenized_texts(np.reshape(model.predict(np.array([test_tensor_fl[random_idx]])), (1, test_tensor.shape[1], test_tensor.shape[2])), wv)[0])

(Test Set) Reconstructed:  ['<s>', 'know', 'hash', 'being', 'back', 'listing', 'text', 'data', 'into', 'textbox', '</s>']
